# Writer: Daeseok Kim

Original code: https://data101.oopy.io/recommendation-engine-cosine-similarity

In [48]:
import pandas as pd
import numpy as np


# csv 파일이되 데이터 구분을 '|' 로 해둔 파일입니다. sep 지정을 안 하면 읽을 수 없습니다.
df = pd.read_csv('seoul.csv', sep=',')  

In [49]:
df

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,23324279,제중건강원,NaN,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,...,1156010600100410002034626,NaN,서울특별시 영등포구 영중로14길 11-17,150035,7250.0,NaN,1,NaN,126.907168,37.520613
1,24525909,민속악기사,NaN,D,소매,D04,취미/오락관련소매,D04A09,악기판매,G47593,...,1120012200101420011000227,NaN,서울특별시 성동구 용답5길 2,133849,4803.0,NaN,NaN,NaN,127.049018,37.566857
2,24715368,태평양진주,NaN,D,소매,D26,시계/귀금속소매,D26A01,시계/귀금속,G47830,...,1111015200100440000000001,금사랑투빌딩,서울특별시 종로구 서순라길 17-10,110390,3138.0,NaN,NaN,NaN,126.993530,37.571848
3,15554136,김선희꼼꼼국어교습소,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,1171011100101970003017195,NaN,서울특별시 송파구 위례성대로12길 31,138834,5640.0,NaN,2,NaN,127.121520,37.510967
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346224,18466695,신내역시티부동산공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,...,1126010500103150000000004,NaN,서울특별시 중랑구 봉화산로59길 25,131800,2057.0,NaN,NaN,NaN,127.103704,37.611201
346225,18455989,석정칼국수,NaN,Q,음식,Q04,분식,Q04A02,국수/만두/칼국수,I56194,...,1126010100102260010031461,NaN,서울특별시 중랑구 면목로 446,131817,2160.0,NaN,1,NaN,127.086895,37.591754
346226,18454507,서울왕도숯불갈비,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1123010300104380010033192,NaN,서울특별시 동대문구 약령시로 122-3,130862,2573.0,NaN,NaN,NaN,127.043357,37.582867
346227,18447435,칠교,NaN,F,생활서비스,F13,기타서비스업,F13A01,사업경영상담,M71531,...,1154510100106870000000003,대륭21차,서울특별시 금천구 범안로 1126,153803,8595.0,NaN,NaN,NaN,126.887062,37.465200


In [59]:
# 음식점 데이터만 쓸 겁니다
df2 = df.loc[df['상권업종대분류명'] == '음식']  


# 다음과 같은 칼럼만 있으면 됩니다
df3 = df2[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

# 그 중에서도 흑석동과 상도1동만 쓸 겁니다.

df_actual = df3.loc[(df3['행정동명'] == '성수1가1동') | (df3['행정동명'] == '성수1가2동') | 
              (df3['행정동명'] == '성수2가1동') | (df3['행정동명'] == '성수2가3동')]
df_actual

,상호명,상권업종중분류명,상권업종소분류명,표준산업분류명,행정동명,위도,경도
124,우영푸드빌,한식,한식/백반/한정식,한식 음식점업,성수2가3동,37.543109,127.059595
1446,원할머니보쌈,한식,족발/보쌈전문,한식 음식점업,성수2가1동,37.538206,127.058366
1476,맛보래,한식,한식/백반/한정식,한식 음식점업,성수2가3동,37.545273,127.056433
2161,철규네분식,분식,라면김밥분식,분식 및 김밥 전문점,성수2가1동,37.537882,127.053935
2548,남도곰탕,한식,설렁탕집,한식 음식점업,성수2가1동,37.537728,127.051595
...,...,...,...,...,...,...,...
345144,캐스터네츠,양식,정통양식/경양식,서양식 음식점업,성수1가1동,37.545668,127.042893
345405,청담순대국,한식,한식/백반/한정식,한식 음식점업,성수1가2동,37.545869,127.048753
345649,오스테리아로에로,양식,정통양식/경양식,서양식 음식점업,성수2가1동,37.539488,127.056381
345722,아니마토,한식,한식/백반/한정식,한식 음식점업,성수1가2동,37.546947,127.043828


In [61]:
df_actual.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]
df_actual

,name,cate_1,cate_2,cate_3,dong,lon,lat
124,우영푸드빌,한식,한식/백반/한정식,한식 음식점업,성수2가3동,37.543109,127.059595
1446,원할머니보쌈,한식,족발/보쌈전문,한식 음식점업,성수2가1동,37.538206,127.058366
1476,맛보래,한식,한식/백반/한정식,한식 음식점업,성수2가3동,37.545273,127.056433
2161,철규네분식,분식,라면김밥분식,분식 및 김밥 전문점,성수2가1동,37.537882,127.053935
2548,남도곰탕,한식,설렁탕집,한식 음식점업,성수2가1동,37.537728,127.051595
...,...,...,...,...,...,...,...
345144,캐스터네츠,양식,정통양식/경양식,서양식 음식점업,성수1가1동,37.545668,127.042893
345405,청담순대국,한식,한식/백반/한정식,한식 음식점업,성수1가2동,37.545869,127.048753
345649,오스테리아로에로,양식,정통양식/경양식,서양식 음식점업,성수2가1동,37.539488,127.056381
345722,아니마토,한식,한식/백반/한정식,한식 음식점업,성수1가2동,37.546947,127.043828


In [63]:
df_actual['cate_mix'] = df_actual['cate_1'] + df_actual['cate_2'] +  df_actual['cate_3']
df_actual['cate_mix'] = df_actual['cate_mix'].str.replace(",", " ")

/var/folders/4d/48ljcj8n3jl31y2pwj395zp00000gn/T/ipykernel_14575/2441054766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['cate_mix'] = df_actual['cate_1'] + df_actual['cate_2'] +  df_actual['cate_3']
/var/folders/4d/48ljcj8n3jl31y2pwj395zp00000gn/T/ipykernel_14575/2441054766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['cate_mix'] = df_actual['cate_mix'].str.replace(",", " ")


In [64]:
pip install sklearn


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [65]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Note: you may need to restart the kernel to use updated packages.


In [66]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df_actual['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]